<a href="https://colab.research.google.com/github/nmq443/cognitive-science-final-project/blob/quang-branch/torcheeg_atcnet-with_preprocessed_data_cwt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# if run on colab
!pip install torcheeg
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# if run on kaggle
!pip install torcheeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.5/214.5 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.8/230.8 kB 15.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.2/107.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.5 MB/s eta 0:00:00
  Created wheel for torcheeg: filename=torc

In [3]:
import torcheeg
from torcheeg import transforms
from torcheeg.datasets import BCICIV2aDataset
from torcheeg.model_selection import KFoldGroupbyTrial
from torch.utils.data import DataLoader
from torcheeg.models import ATCNet, EEGNet
import torch

from torcheeg.trainers import ClassifierTrainer

import pytorch_lightning as pl

In [4]:
# if run on kaggle
# root_data_path = '/kaggle/input/bci-competition-iv-dataset-2a-in-mat-format/BCICIV-2a-mat'

# if run on colab
root_data_path = '/content/drive/MyDrive/BCICIV-2a-mat'

# if run on local machine
# root_data_path = './BCICIV-2a-mat/'

In [9]:
dataset = BCICIV2aDataset(
    root_path=root_data_path,
    io_path=f'./examples_pipeline/bciciv-2a',
    online_transform=transforms.Compose([
        transforms.CWTSpectrum(),
        transforms.ToTensor(),
    ]),
    label_transform=transforms.Compose([
        transforms.Select('label'),
        transforms.Lambda(lambda x: x - 1)
    ]),
    chunk_size=7*250,
    num_worker=2
)

[2024-05-21 11:38:27] INFO (torcheeg/MainThread) 🔍 | Detected cached processing results, reading cache from ./examples_pipeline/bciciv-2a.
INFO:torcheeg:🔍 | Detected cached processing results, reading cache from ./examples_pipeline/bciciv-2a.


In [6]:
print("Dataset's info: ")
print(dataset.info)

Dataset's info: 
      start_at  end_at   clip_id subject_id  trial_id session subject  run  \
0          251    2001    A04E_0        A04         0       E     A04    3   
1         2254    4004    A04E_1        A04         1       E     A04    3   
2         4172    5922    A04E_2        A04         2       E     A04    3   
3         6124    7874    A04E_3        A04         3       E     A04    3   
4         8132    9882    A04E_4        A04         4       E     A04    3   
...        ...     ...       ...        ...       ...     ...     ...  ...   
5179     86751   88501  A08T_283        A08        43       T     A08    8   
5180     88657   90407  A08T_284        A08        44       T     A08    8   
5181     90585   92335  A08T_285        A08        45       T     A08    8   
5182     92699   94449  A08T_286        A08        46       T     A08    8   
5183     94758   96508  A08T_287        A08        47       T     A08    8   

      label  _record_id  
0         4   _recor

In [10]:
k_fold = KFoldGroupbyTrial(
    n_splits=10,
    split_path='./examples_pipeline/split',
    shuffle=True,
    random_state=44
)

In [11]:
DEVICE = 'gpu' if torch.cuda.is_available() else 'cpu'

for i, (train_dataset, val_dataset) in enumerate(k_fold.split(dataset)):
    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=64,
        shuffle=True,
        num_workers=4
    )
    val_loader = DataLoader(
        dataset=val_dataset,
        batch_size=64,
        shuffle=False,
        num_workers=4
    )

    model = ATCNet(
        num_classes=4,
        num_electrodes=22,
        chunk_size=7*250,
    )

    trainer = ClassifierTrainer(
        model=model,
        num_classes=4,
        lr=1e-4,
        weight_decay=1e-4,
        accelerator=DEVICE
    )

    trainer.fit(
        train_loader,
        val_loader,
        max_epochs=50,
        default_root_dir=f'./examples_pipeline/atcnet_model/{i}',
        callbacks=[pl.callbacks.ModelCheckpoint(save_last=True)],
        enable_progress_bar=True,
        enable_model_summary=True,
        limit_val_batches=0.0
    )

    score = trainer.test(
        val_loader,
        enable_progress_bar=True,
        enable_model_summary=True
    )[0]
    print(f"Fold {i} test accuracy: {score['test_accuracy']: .4f}")

[2024-05-21 11:38:40] INFO (torcheeg/MainThread) 📊 | Detected existing split of train and test set, use existing split from ./examples_pipeline/split.
INFO:torcheeg:📊 | Detected existing split of train and test set, use existing split from ./examples_pipeline/split.
[2024-05-21 11:38:40] INFO (torcheeg/MainThread) 💡 | If the dataset is re-generated, you need to re-generate the split of the dataset instead of using the previous split.
INFO:torcheeg:💡 | If the dataset is re-generated, you need to re-generate the split of the dataset instead of using the previous split.
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if

Training: |          | 0/? [00:00<?, ?it/s]

RuntimeError: DataLoader worker (pid(s) 2939, 2943, 2945) exited unexpectedly

In [ ]:
!mkdir weights

In [ ]:
atc_weights_path = './weights/atc_weights.pt'
torch.save(model.state_dict(), atc_weights_path)
# to load weight:
# model.load_state_dict(torch.load(atc_weights_path))